In [37]:
import requests

BASE_URL = "https://tienda.mercadona.es/api/"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/118.0 Safari/537.36",
    "Accept": "application/json"
}

def fetch_categories():
    url = f"{BASE_URL}categories/"
    resp = requests.get(url, headers=HEADERS, timeout=10)
    if resp.status_code == 200:
        return resp.json().get("results", [])
    else:
        print(f"Error {resp.status_code} al obtener categorías")
        return []

def print_category_names(categories, level=0):
    """Imprime recursivamente los nombres de categorías y subcategorías"""
    for cat in categories:
        print("  " * level + f"- {cat.get('name', 'sin nombre')}")
        if "categories" in cat and cat["categories"]:
            print_category_names(cat["categories"], level + 1)

if __name__ == "__main__":
    categories = fetch_categories()
    if categories:
        print("Categorías de Mercadona:")
        print_category_names(categories)
    else:
        print("No se obtuvieron categorías.")


Categorías de Mercadona:
- Aceite, especias y salsas
  - Aceite, vinagre y sal
  - Especias
  - Mayonesa, ketchup y mostaza
  - Otras salsas
- Agua y refrescos
  - Agua
  - Isotónico y energético
  - Refresco de cola
  - Refresco de naranja y de limón
  - Tónica y bitter
  - Refresco de té y sin gas
- Aperitivos
  - Aceitunas y encurtidos
  - Frutos secos y fruta desecada
  - Patatas fritas y snacks
- Arroz, legumbres y pasta
  - Arroz
  - Legumbres
  - Pasta y fideos
- Azúcar, caramelos y chocolate
  - Azúcar y edulcorante
  - Chicles y caramelos
  - Chocolate
  - Golosinas
  - Mermelada y miel
  - Turrones
- Bebé
  - Alimentación infantil
  - Biberón y chupete
  - Higiene y cuidado
  - Toallitas y pañales
- Bodega
  - Cerveza
  - Cerveza sin alcohol
  - Licores
  - Sidra y cava
  - Tinto de verano y sangría
  - Vino blanco
  - Vino lambrusco y espumoso
  - Vino rosado
  - Vino tinto
- Cacao, café e infusiones
  - Cacao soluble y chocolate a la taza
  - Café cápsula y monodosis
  - Ca

In [39]:
import requests
import csv

BASE_URL = "https://tienda.mercadona.es/api/"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/118.0 Safari/537.36",
    "Accept": "application/json"
}

def fetch_categories():
    url = f"{BASE_URL}categories/"
    resp = requests.get(url, headers=HEADERS, timeout=10)
    if resp.status_code == 200:
        return resp.json().get("results", [])
    else:
        print(f"Error {resp.status_code} al obtener categorías")
        return []

def fetch_products(category_id):
    """Obtiene todos los productos de una categoría"""
    url = f"{BASE_URL}categories/{category_id}/products/"
    resp = requests.get(url, headers=HEADERS, timeout=10)
    if resp.status_code == 200:
        return resp.json().get("results", [])
    else:
        print(f"Error {resp.status_code} al obtener productos de categoría {category_id}")
        return []

def process_category(cat, csv_writer):
    """Procesa la categoría, sus subcategorías y guarda productos en CSV"""
    # Guardar productos de esta categoría
    products = fetch_products(cat["id"])
    for prod in products:
        csv_writer.writerow({
            "category_id": cat["id"],
            "category_name": cat["name"],
            "product_id": prod.get("id", ""),
            "product_name": prod.get("name", ""),
            "price": prod.get("price", "")
        })

    # Recursivamente procesar subcategorías
    if "categories" in cat and cat["categories"]:
        for subcat in cat["categories"]:
            process_category(subcat, csv_writer)

if __name__ == "__main__":
    categories = fetch_categories()
    if categories:
        with open("productos_mercadona.csv", "w", newline="", encoding="utf-8") as csvfile:
            fieldnames = ["category_id", "category_name", "product_id", "product_name", "price"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            
            for cat in categories:
                process_category(cat, writer)
        print("CSV generado correctamente: productos_mercadona.csv")
    else:
        print("No se obtuvieron categorías.")


Error 404 al obtener productos de categoría 12
Error 404 al obtener productos de categoría 112
Error 404 al obtener productos de categoría 115
Error 404 al obtener productos de categoría 116
Error 404 al obtener productos de categoría 117
Error 404 al obtener productos de categoría 18
Error 404 al obtener productos de categoría 156
Error 404 al obtener productos de categoría 163
Error 404 al obtener productos de categoría 158
Error 404 al obtener productos de categoría 159
Error 404 al obtener productos de categoría 161
Error 404 al obtener productos de categoría 162
Error 404 al obtener productos de categoría 15
Error 404 al obtener productos de categoría 135
Error 404 al obtener productos de categoría 133
Error 404 al obtener productos de categoría 132
Error 404 al obtener productos de categoría 13
Error 404 al obtener productos de categoría 118
Error 404 al obtener productos de categoría 121
Error 404 al obtener productos de categoría 120
Error 404 al obtener productos de categoría 

In [40]:
import requests
import csv
import time

# --- Constantes y Configuración ---
BASE_URL = "https://tienda.mercadona.es/api/"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/118.0.0.0 Safari/537.36",
    "Accept": "application/json"
}
CSV_FILE = "productos_mercadona.csv"
PRODUCT_IDS_SEEN = set()  # Para evitar guardar el mismo producto varias veces
CATEGORY_PATH = []  # Para rastrear la ruta completa de la categoría

# --- Funciones de Utilidad de API ---

def fetch_data(url):
    """Realiza una petición GET a la URL y maneja errores."""
    try:
        # Añadimos un pequeño retraso (delay) entre peticiones para ser corteses con el servidor.
        time.sleep(0.5)
        resp = requests.get(url, headers=HEADERS, timeout=10)
        resp.raise_for_status() # Lanza una excepción para códigos de estado de error (4xx o 5xx)
        return resp.json()
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener datos de {url}: {e}")
        return None

def fetch_categories():
    """Obtiene la lista de categorías principales."""
    url = f"{BASE_URL}categories/"
    data = fetch_data(url)
    return data.get("results", []) if data else []

def fetch_category_products(category_id):
    """Obtiene los detalles y productos para un ID de categoría específico."""
    url = f"{BASE_URL}categories/{category_id}/"
    data = fetch_data(url)
    return data.get("products", []) if data else []

# --- Función de Procesamiento y CSV ---

def write_product_to_csv(writer, product, category_path_str):
    """Extrae los campos relevantes del producto y los escribe en el archivo CSV."""
    
    # Intenta obtener el precio de la estructura 'price_instructions'
    price_info = product.get('price_instructions', {})
    price = price_info.get('unit_price')
    unit_price = price_info.get('bulk_price')
    reference_price = price_info.get('reference_price')
    reference_format = price_info.get('reference_format')

    product_data = {
        "id": product.get("id"),
        "nombre": product.get("display_name"),
        "subtitulo": product.get("packaging"),
        "precio_unidad": price,
        "precio_unidad_por": unit_price,
        "precio_referencia": f"{reference_price} {reference_format}".strip(),
        "ruta_categoria": category_path_str
    }
    
    writer.writerow(product_data)


def traverse_categories_and_save(categories, writer, category_path):
    """Recorre recursivamente las categorías, extrae los productos y los guarda."""
    for cat in categories:
        cat_id = cat.get('id')
        cat_name = cat.get('name')
        
        if not cat_id or not cat_name:
            continue

        # 1. Actualiza la ruta de la categoría actual
        current_category_path = category_path + [cat_name]
        category_path_str = " > ".join(current_category_path)
        
        print(f"-> Procesando Categoría: {category_path_str} (ID: {cat_id})")

        # 2. Obtiene los productos de la categoría/subcategoría
        products = fetch_category_products(cat_id)
        
        # 3. Itera sobre los productos y los guarda
        for product in products:
            product_id = product.get('id')
            if product_id and product_id not in PRODUCT_IDS_SEEN:
                write_product_to_csv(writer, product, category_path_str)
                PRODUCT_IDS_SEEN.add(product_id)

        # 4. Llama recursivamente a las subcategorías
        if "categories" in cat and cat["categories"]:
            traverse_categories_and_save(cat["categories"], writer, current_category_path)


# --- Ejecución Principal ---

if __name__ == "__main__":
    print(f"Iniciando la extracción de productos de Mercadona y guardando en {CSV_FILE}...")
    
    # Definición de las cabeceras (headers) del CSV
    fieldnames = [
        "id", 
        "nombre", 
        "subtitulo", 
        "precio_unidad", 
        "precio_unidad_por", 
        "precio_referencia",
        "ruta_categoria"
    ]
    
    try:
        # Abrimos el archivo CSV y lo inicializamos con las cabeceras
        with open(CSV_FILE, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
            writer.writeheader()

            # 1. Obtiene las categorías principales
            top_level_categories = fetch_categories()
            
            if top_level_categories:
                # 2. Inicia el recorrido recursivo
                traverse_categories_and_save(top_level_categories, writer, [])
                print(f"\n✅ Proceso completado. Total de productos únicos guardados: {len(PRODUCT_IDS_SEEN)}")
            else:
                print("❌ No se pudieron obtener categorías para comenzar.")
                
    except Exception as e:
        print(f"\n❌ Ocurrió un error inesperado durante la escritura del archivo: {e}")

Iniciando la extracción de productos de Mercadona y guardando en productos_mercadona.csv...
-> Procesando Categoría: Aceite, especias y salsas (ID: 12)
Error al obtener datos de https://tienda.mercadona.es/api/categories/12/: 404 Client Error: Not Found for url: https://tienda.mercadona.es/api/categories/12/
-> Procesando Categoría: Aceite, especias y salsas > Aceite, vinagre y sal (ID: 112)
-> Procesando Categoría: Aceite, especias y salsas > Especias (ID: 115)
-> Procesando Categoría: Aceite, especias y salsas > Mayonesa, ketchup y mostaza (ID: 116)
-> Procesando Categoría: Aceite, especias y salsas > Otras salsas (ID: 117)
-> Procesando Categoría: Agua y refrescos (ID: 18)
Error al obtener datos de https://tienda.mercadona.es/api/categories/18/: 404 Client Error: Not Found for url: https://tienda.mercadona.es/api/categories/18/
-> Procesando Categoría: Agua y refrescos > Agua (ID: 156)
-> Procesando Categoría: Agua y refrescos > Isotónico y energético (ID: 163)
-> Procesando Categor

In [41]:
import requests
import time

# --- Constantes y Configuración ---
BASE_URL = "https://tienda.mercadona.es/api/"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/118.0.0.0 Safari/537.36",
    "Accept": "application/json"
}
PRODUCT_IDS_SEEN = set()  # Para evitar procesar el mismo producto varias veces

# --- Funciones de Utilidad de API ---

def fetch_data(url):
    """Realiza una petición GET a la URL y maneja errores."""
    try:
        # Pequeño delay entre peticiones.
        time.sleep(0.5)
        resp = requests.get(url, headers=HEADERS, timeout=10)
        resp.raise_for_status() # Lanza una excepción para códigos de estado de error (4xx o 5xx)
        return resp.json()
    except requests.exceptions.RequestException as e:
        # Imprime la URL y el error para facilitar el debugging
        print(f"❌ Error al obtener datos de {url}: {e}")
        return None

def fetch_categories():
    """Obtiene la lista de categorías principales."""
    url = f"{BASE_URL}categories/"
    data = fetch_data(url)
    return data.get("results", []) if data else []

def fetch_category_products(category_id):
    """Obtiene los detalles y productos para un ID de categoría específico."""
    # Este es el endpoint clave: /api/categories/{category_id}/
    url = f"{BASE_URL}categories/{category_id}/"
    data = fetch_data(url)
    # Devolvemos el listado de productos
    return data.get("products", []) if data else []

# --- Función de Impresión de Detalles ---

def print_product_details(product, category_path_str):
    """Extrae los campos relevantes del producto y los imprime en la consola."""
    
    # Intenta obtener el precio de la estructura 'price_instructions'
    price_info = product.get('price_instructions', {})
    price = price_info.get('unit_price')
    reference_price = price_info.get('reference_price')
    reference_format = price_info.get('reference_format')
    
    # Construcción de la línea de output
    output = (
        f"  - ID: {product.get('id')} | "
        f"Nombre: **{product.get('display_name')}** | "
        f"Precio: {price} | "
        f"Precio Ref.: {reference_price} {reference_format} | "
        f"Categoría: {category_path_str}"
    )
    print(output)


def traverse_categories_and_print(categories, category_path):
    """Recorre recursivamente las categorías y subcategorías."""
    for cat in categories:
        cat_id = cat.get('id')
        cat_name = cat.get('name')
        
        if not cat_id or not cat_name:
            continue

        # 1. Define la ruta de la categoría
        current_category_path = category_path + [cat_name]
        category_path_str = " > ".join(current_category_path)
        
        print(f"\n📢 Procesando Categoría: **{category_path_str}** (ID: {cat_id})")

        # 2. Obtiene los productos de la categoría/subcategoría
        products = fetch_category_products(cat_id)
        
        if products:
            print(f"  🛒 Se encontraron {len(products)} productos. Detalles:")
        else:
            print("  ⚠️ No se encontraron productos.")

        # 3. Itera sobre los productos e imprime sus detalles
        for product in products:
            product_id = product.get('id')
            if product_id and product_id not in PRODUCT_IDS_SEEN:
                print_product_details(product, category_path_str)
                PRODUCT_IDS_SEEN.add(product_id)

        # 4. Llama recursivamente a las subcategorías
        if "categories" in cat and cat["categories"]:
            traverse_categories_and_print(cat["categories"], current_category_path)


# --- Ejecución Principal ---

if __name__ == "__main__":
    print("Iniciando la extracción de productos de Mercadona y mostrando en consola...")
    
    # 1. Obtiene las categorías principales
    top_level_categories = fetch_categories()
    
    if top_level_categories:
        # 2. Inicia el recorrido recursivo
        traverse_categories_and_print(top_level_categories, [])
        print(f"\n✅ Proceso de impresión completado. Total de productos únicos procesados: {len(PRODUCT_IDS_SEEN)}")
    else:
        print("❌ No se pudieron obtener categorías principales para comenzar.")

Iniciando la extracción de productos de Mercadona y mostrando en consola...

📢 Procesando Categoría: **Aceite, especias y salsas** (ID: 12)
❌ Error al obtener datos de https://tienda.mercadona.es/api/categories/12/: 404 Client Error: Not Found for url: https://tienda.mercadona.es/api/categories/12/
  ⚠️ No se encontraron productos.

📢 Procesando Categoría: **Aceite, especias y salsas > Aceite, vinagre y sal** (ID: 112)
  ⚠️ No se encontraron productos.

📢 Procesando Categoría: **Aceite, especias y salsas > Especias** (ID: 115)
  ⚠️ No se encontraron productos.

📢 Procesando Categoría: **Aceite, especias y salsas > Mayonesa, ketchup y mostaza** (ID: 116)
  ⚠️ No se encontraron productos.

📢 Procesando Categoría: **Aceite, especias y salsas > Otras salsas** (ID: 117)
  ⚠️ No se encontraron productos.

📢 Procesando Categoría: **Agua y refrescos** (ID: 18)
❌ Error al obtener datos de https://tienda.mercadona.es/api/categories/18/: 404 Client Error: Not Found for url: https://tienda.mercado

In [42]:
import requests
import csv
import time

# --- Constantes y Configuración ---
BASE_URL = "https://tienda.mercadona.es/api/"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/118.0.0.0 Safari/537.36",
    "Accept": "application/json"
}
CSV_FILE = "productos_mercadona.csv"
PRODUCT_IDS_SEEN = set()  # Usamos un set para almacenar IDs de productos únicos
FIELDNAMES = [
    "id", 
    "nombre", 
    "subtitulo", 
    "precio_unidad", 
    "precio_ref_valor", 
    "precio_ref_formato",
    "ruta_categoria"
]

# --- Funciones de Utilidad de API ---

def fetch_data(url):
    """Realiza una petición GET a la URL y maneja errores."""
    try:
        # Pausa de 0.5s para no sobrecargar la API (buena práctica de web scraping)
        time.sleep(0.5) 
        resp = requests.get(url, headers=HEADERS, timeout=10)
        resp.raise_for_status() # Lanza un HTTPError si el estado es 4xx o 5xx
        return resp.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ Error al obtener datos de {url}: {e}")
        return None

def fetch_categories():
    """Obtiene la lista de categorías principales."""
    url = f"{BASE_URL}categories/"
    data = fetch_data(url)
    return data.get("results", []) if data else []

def fetch_category_products(category_id):
    """Obtiene los detalles y productos para un ID de categoría específico."""
    # Este es el endpoint clave: /api/categories/{category_id}/
    url = f"{BASE_URL}categories/{category_id}/"
    data = fetch_data(url)
    return data.get("products", []) if data else []

# --- Función de Procesamiento y CSV ---

def extract_and_save_product(writer, product, category_path_str):
    """Extrae los campos relevantes del producto y los escribe en el archivo CSV."""
    
    # Intenta obtener el precio de la estructura 'price_instructions'
    price_info = product.get('price_instructions', {})
    
    # Preparamos el diccionario con los datos del producto
    product_data = {
        "id": product.get("id", "N/A"),
        "nombre": product.get("display_name", "N/A"),
        "subtitulo": product.get("packaging", "N/A"),
        "precio_unidad": price_info.get('unit_price', "N/A"),
        "precio_ref_valor": price_info.get('reference_price', "N/A"),
        "precio_ref_formato": price_info.get('reference_format', "N/A"),
        "ruta_categoria": category_path_str
    }
    
    # Escribimos la fila en el CSV
    writer.writerow(product_data)


def traverse_categories_and_save(categories, writer, category_path):
    """Recorre recursivamente las categorías, extrae los productos y los guarda."""
    global PRODUCT_IDS_SEEN # Necesario para modificar el set global

    for cat in categories:
        cat_id = cat.get('id')
        cat_name = cat.get('name')
        
        if not cat_id or not cat_name:
            continue

        # 1. Define la ruta de la categoría actual
        current_category_path = category_path + [cat_name]
        category_path_str = " > ".join(current_category_path)
        
        print(f"-> Procesando: {category_path_str} (ID: {cat_id})")

        # 2. Obtiene los productos de la categoría/subcategoría
        products = fetch_category_products(cat_id)
        
        # 3. Itera sobre los productos y los guarda si son únicos
        for product in products:
            product_id = product.get('id')
            if product_id and product_id not in PRODUCT_IDS_SEEN:
                extract_and_save_product(writer, product, category_path_str)
                PRODUCT_IDS_SEEN.add(product_id)
        
        # 4. Llama recursivamente a las subcategorías
        if "categories" in cat and cat["categories"]:
            traverse_categories_and_save(cat["categories"], writer, current_category_path)


# --- Ejecución Principal ---

if __name__ == "__main__":
    print(f"Iniciando la extracción de productos de Mercadona y guardando en {CSV_FILE}...")
    
    try:
        # Abrimos el archivo CSV en modo escritura, especificamos 'utf-8' y sin líneas en blanco
        with open(CSV_FILE, 'w', newline='', encoding='utf-8') as csvfile:
            # Inicializamos el DictWriter con el delimitador ';'
            writer = csv.DictWriter(csvfile, fieldnames=FIELDNAMES, delimiter=';')
            writer.writeheader() # Escribimos la cabecera
            
            # 1. Obtiene las categorías principales
            top_level_categories = fetch_categories()
            
            if top_level_categories:
                # 2. Inicia el recorrido recursivo
                traverse_categories_and_save(top_level_categories, writer, [])
                print(f"\n✅ Proceso completado. Total de productos únicos guardados: {len(PRODUCT_IDS_SEEN)}")
            else:
                print("❌ No se obtuvieron categorías. Verifica la conexión o el BASE_URL.")
                
    except Exception as e:
        print(f"\n❌ Ocurrió un error inesperado durante la ejecución: {e}")

Iniciando la extracción de productos de Mercadona y guardando en productos_mercadona.csv...
-> Procesando: Aceite, especias y salsas (ID: 12)
❌ Error al obtener datos de https://tienda.mercadona.es/api/categories/12/: 404 Client Error: Not Found for url: https://tienda.mercadona.es/api/categories/12/
-> Procesando: Aceite, especias y salsas > Aceite, vinagre y sal (ID: 112)
-> Procesando: Aceite, especias y salsas > Especias (ID: 115)
-> Procesando: Aceite, especias y salsas > Mayonesa, ketchup y mostaza (ID: 116)
-> Procesando: Aceite, especias y salsas > Otras salsas (ID: 117)
-> Procesando: Agua y refrescos (ID: 18)
❌ Error al obtener datos de https://tienda.mercadona.es/api/categories/18/: 404 Client Error: Not Found for url: https://tienda.mercadona.es/api/categories/18/
-> Procesando: Agua y refrescos > Agua (ID: 156)
-> Procesando: Agua y refrescos > Isotónico y energético (ID: 163)
-> Procesando: Agua y refrescos > Refresco de cola (ID: 158)
-> Procesando: Agua y refrescos > Re

In [43]:
import requests
import csv
import time

# --- Constantes y Configuración ---
BASE_URL = "https://tienda.mercadona.es/api/"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/118.0.0.0 Safari/537.36",
    "Accept": "application/json"
}
CSV_FILE = "productos_mercadona.csv"
PRODUCT_IDS_SEEN = set()  # Para evitar duplicados
FIELDNAMES = [
    "id", 
    "nombre", 
    "subtitulo", 
    "precio_unidad", 
    "precio_ref_valor", 
    "precio_ref_formato",
    "ruta_categoria"
]

# --- Funciones de Utilidad de API ---

def fetch_data(url):
    """Realiza una petición GET a la URL y maneja errores, con delay."""
    try:
        time.sleep(0.5) 
        resp = requests.get(url, headers=HEADERS, timeout=10)
        resp.raise_for_status()
        return resp.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ Error al obtener datos de {url}: {e}")
        return None

# --- Función de Procesamiento y CSV ---

def extract_and_save_product(writer, product, category_path_str):
    """Extrae los campos relevantes del producto y los escribe en el archivo CSV si es nuevo."""
    global PRODUCT_IDS_SEEN
    product_id = product.get('id')
    
    if product_id and product_id not in PRODUCT_IDS_SEEN:
        price_info = product.get('price_instructions', {})
        
        product_data = {
            "id": product_id,
            "nombre": product.get("display_name", "N/A"),
            "subtitulo": product.get("packaging", "N/A"),
            "precio_unidad": price_info.get('unit_price', "N/A"),
            "precio_ref_valor": price_info.get('reference_price', "N/A"),
            "precio_ref_formato": price_info.get('reference_format', "N/A"),
            "ruta_categoria": category_path_str
        }
        
        writer.writerow(product_data)
        PRODUCT_IDS_SEEN.add(product_id)
        # Puedes descomentar la siguiente línea para depuración:
        # print(f"  🛒 Guardado: {product_data['nombre']} | Ruta: {category_path_str}")


def traverse_category(category_info, writer, category_path):
    """
    Recorre recursivamente una categoría y sus subcategorías. 
    Lógica adaptada de PHP: extrae productos anidados Y llama a la siguiente categoría.
    """

    cat_id = category_info.get('id')
    cat_name = category_info.get('name')
    
    if not cat_id or not cat_name:
        return

    # 1. Actualiza la ruta de la categoría actual
    current_category_path = category_path + [cat_name]
    category_path_str = " > ".join(current_category_path)
    
    print(f"-> Procesando: {category_path_str} (ID: {cat_id})")

    # 2. **EXTRAER PRODUCTOS ANIDADOS** (Si la categoría actual ya los trae)
    # Esto ocurre con frecuencia en las respuestas de la API de Mercadona.
    if 'products' in category_info and category_info['products']:
        for product in category_info['products']:
            extract_and_save_product(writer, product, category_path_str)


    # 3. **Obtener el DETALLE COMPLETO DE LA CATEGORÍA** para explorar subniveles
    # La API requiere una segunda llamada para obtener la lista completa de subcategorías
    # o, si es un nivel hoja, para listar todos sus productos.
    category_detail = fetch_data(f"{BASE_URL}categories/{cat_id}/")
    
    if not category_detail:
        return # Salir si la llamada falló

    # 4. **EXTRAER PRODUCTOS DEL DETALLE** (Si es una categoría final/hoja)
    # Si la categoría consultada tiene un listado de productos, también los guardamos.
    if 'products' in category_detail and category_detail['products']:
        for product in category_detail['products']:
            extract_and_save_product(writer, product, category_path_str)
            
    # 5. Llamada recursiva a las subcategorías (categories)
    # Almacenadas en el detalle de la categoría
    if 'categories' in category_detail and category_detail['categories']:
        for sub_cat in category_detail['categories']:
            # Lógica de Backtracking implícita al pasar 'current_category_path'
            traverse_category(sub_cat, writer, current_category_path)


def start_scraping(writer):
    """Inicia el proceso de extracción obteniendo las categorías principales."""
    
    # Obtener categorías de nivel superior
    top_level_data = fetch_data(f"{BASE_URL}categories/")
    
    if not top_level_data or not top_level_data.get('results'):
        print("❌ No se pudieron obtener categorías principales para comenzar.")
        return

    # Iniciar el recorrido por cada categoría principal
    for category in top_level_data['results']:
        traverse_category(category, writer, [])


# --- Ejecución Principal ---

if __name__ == "__main__":
    print(f"Iniciando la extracción de productos de Mercadona y guardando en {CSV_FILE}...")
    
    try:
        # Abrimos el archivo CSV
        with open(CSV_FILE, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=FIELDNAMES, delimiter=';')
            writer.writeheader() 
            
            start_scraping(writer)
            
            print(f"\n✅ Proceso completado. Total de productos únicos guardados: {len(PRODUCT_IDS_SEEN)}")
                
    except Exception as e:
        print(f"\n❌ Ocurrió un error inesperado durante la escritura del archivo: {e}")

Iniciando la extracción de productos de Mercadona y guardando en productos_mercadona.csv...
-> Procesando: Aceite, especias y salsas (ID: 12)
❌ Error al obtener datos de https://tienda.mercadona.es/api/categories/12/: 404 Client Error: Not Found for url: https://tienda.mercadona.es/api/categories/12/
-> Procesando: Agua y refrescos (ID: 18)
❌ Error al obtener datos de https://tienda.mercadona.es/api/categories/18/: 404 Client Error: Not Found for url: https://tienda.mercadona.es/api/categories/18/
-> Procesando: Aperitivos (ID: 15)
❌ Error al obtener datos de https://tienda.mercadona.es/api/categories/15/: 404 Client Error: Not Found for url: https://tienda.mercadona.es/api/categories/15/
-> Procesando: Arroz, legumbres y pasta (ID: 13)
❌ Error al obtener datos de https://tienda.mercadona.es/api/categories/13/: 404 Client Error: Not Found for url: https://tienda.mercadona.es/api/categories/13/
-> Procesando: Azúcar, caramelos y chocolate (ID: 9)
❌ Error al obtener datos de https://tien